In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "admin"
password = "admin"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')


app.layout = html.Div([
   
   html.Div(id='hidden-div', style={'display':'none'}),
      
    
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
    dcc.Dropdown(
        id='dropdown',
        options=[
            {'label': 'Water Rescue', 'value':'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value':'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value':'Disaster or Individual Tracking'},
            {'label': 'Reset', 'value':'Reset'}
        ],
        value = 'Reset',
        
    ),
    
),
    html.Hr(),
   dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable = False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
        
    ),

       
    
    html.Br(),
    html.Hr(),
    html.Div(className='inputs',
         style={'display' : 'flex',
                'margin-right':0},
             children=[
         html.Div(
    [
        dcc.Input(id="row", type="number", placeholder="Input row"),
        dcc.Input(
            id="lat", type="number", placeholder="Latitude"
        ),
        dcc.Input(
            id="long", type="number"
            , placeholder="Longitude",
        ),
        html.Hr(),
        html.Div(id="number-out"),
    ]
             )]),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    html.Div([
    html.H6('Created By Corey Wright'),
    html.A([
            html.Img(
                src='https://learn.snhu.edu/content/enforced/851421-CS-340-T1035-OL-TRAD-UG.21EW1/course_documents/Grazioso%20Salvare%20Logo.png?_&d2lSessionVal=pix0CjbJEewykxOIfttpni4st&ou=851421',
                style={
                    'height' : '25%',
                    'width' : '25%',
                    'position' : 'relative',
                    'padding-top' : 0,
                    'padding-right' : 0
                })
    ], href='https://www.snhu.edu')
]), 
   dcc.Store(id='row1'),
    dcc.Store(id='lat1'),
    dcc.Store(id='long1'),
])

#############################################
# Interaction Between Components / Controller
#############################################




app

@app.callback(
    Output('datatable-id','data'),
    [Input('dropdown', 'value')]
)
def update_output(value):
    if(value == 'Water Rescue'):
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]}, "sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}})))
    elif(value == 'Mountain or Wilderness Rescue'):
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]}, "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}})))
    elif(value == 'Disaster or Individual Tracking'):
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound", "Rottweiler"]}, "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}})))
    elif(value == 'Reset'):
        df = pd.DataFrame.from_records(shelter.read({}))
    return df.to_dict('records')
        

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    Water = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]}, "sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}})))
    Mountain= pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]}, "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}})))
    Disaster = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":{"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound", "Rottweiler"]}, "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}})))

    return[
        dcc.Graph(
    figure={
    
        'data':[ {'x':['Water','Mountain','Disaster'], 'y':[len(Water),len(Mountain),len(Disaster) ],'type': 'bar', 'name':'SF'}]
    })
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('row','value')])
def update_map(viewData,value):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75957,-97.5524], children=[
                dl.Tooltip(dff.iloc[value,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[value,9])
                ])
            ])
        ])
    ]

app
